<a href="https://colab.research.google.com/github/bewithankit/CS3DP19/blob/main/word_count_using_apache_spark_with_pyspark_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Set Up the Environment:**


*   First, you need to install all the necessary components to run Spark in Colab. This includes Java, Spark itself, and findspark, a Python library that makes it easier to find Spark.

*   Use the `!` operator to run the following commands in a Colab cell to install Java and Spark, and set up the environment variables:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


*   Then, set the environment variables for Java and Spark in Python:



In [41]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"


**2. Initialize Spark:**


*   Import `findspark` and initialize it. Then create a SparkContext.




In [37]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('wordCount')
sc = SparkContext(conf=conf)
# sc = SparkContext.getOrCreate(conf=conf)


**3. Create an RDD:**

* Create a base RDD from a simple list of words and parallelize it.

In [39]:
wordsList = ['blue', 'orange', 'red', 'blue', 'red', 'blue']
wordsRDD = sc.parallelize(wordsList, 5)

# wordsList = sc.textFile("/content/ulysses_short.txt")

print(type(wordsRDD))  # Should print <class 'pyspark.rdd.RDD'>

# Use glom to see how the data is distributed across partitions
glommedRDD = wordsRDD.glom()
print("\n")
print(glommedRDD.collect())

<class 'pyspark.rdd.RDD'>




AttributeError: ignored

**4. Transform the RDD:**

* Use a map operation to transform each word in the RDD.

In [ ]:
# Using a defined function
def webify(x):
    return x+".com"
webified_RDD = wordsRDD.map(webify)
print(webified_RDD.collect())

# Using a lambda function # lambda <args>: <method body>
webified_RDD = wordsRDD.map(lambda word: word + '.com')
print("\n")
print(webified_RDD.collect())


['blue.com', 'orange.com', 'red.com', 'blue.com', 'red.com', 'blue.com']


['blue.com', 'orange.com', 'red.com', 'blue.com', 'red.com', 'blue.com']


**5. Create Pair RDDs:**

* Map each word to a tuple containing the word and the number 1, to prepare for counting.

In [ ]:
wordPairs = wordsRDD.map(lambda word: (word, 1))
print(wordPairs.collect())


[('blue', 1), ('orange', 1), ('red', 1), ('blue', 1), ('red', 1), ('blue', 1)]


**6. Count the Words:**

* Two approaches can be used: `groupByKey()` or `reduceByKey()`. The latter is more efficient.

In [ ]:
# Using groupByKey()
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
  print ('{0}: {1}'.format(key, list(value)))

wordCountsGrouped = wordsGrouped.mapValues(sum)
print(wordCountsGrouped.collect())

# Using reduceByKey()
wordCounts = wordPairs.reduceByKey(lambda x, y: x + y)
print("\n")
print(wordCounts.collect())


orange: [1]
red: [1, 1]
blue: [1, 1, 1]
[('orange', 1), ('red', 2), ('blue', 3)]


[('orange', 1), ('red', 2), ('blue', 3)]


**7. Run the Complete Application:**

* Combine all the steps into a single sequence to count the words.

In [ ]:
wordCountsCollected = (wordsRDD
                       .map(lambda word: (word, 1))
                       .reduceByKey(lambda x, y: x + y)
                       .collect())
print(wordCountsCollected)


[('orange', 1), ('red', 2), ('blue', 3)]


**8. Shutting Down:**

* After you are done, stop the SparkContext to free up resources.

In [45]:
sc.stop()


**Bonus: Using a `.txt` file:**

In [46]:
from pyspark import SparkContext

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('wordCount')
sc = SparkContext(conf=conf)

In [47]:
linesRDD = sc.textFile("/content/ulysses.txt")

wordsRDD = linesRDD.flatMap(lambda line: line.split())

glommedRDD = wordsRDD.glom()
print("\nData partitioning after flatMap:\n", glommedRDD.collect())


Data partitioning after flatMap:
 [['The', 'Project', 'Gutenberg', 'eBook', 'of', 'Ulysses', 'This', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'United', 'States', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever.', 'You', 'may', 'copy', 'it,', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'Project', 'Gutenberg', 'License', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www.gutenberg.org.', 'If', 'you', 'are', 'not', 'located', 'in', 'the', 'United', 'States,', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'eBook.', 'Title:', 'Ulysses', 'Author:', 'James', 'Joyce', 'Release', 'date:', 'July', '1,', '2003', '[eBook', '#4300]', 'Most', 'recently', 'updated:', 'March', '1,', '2021', 'Language:', 'English', 'Credits:', 'EBook', 'prod

In [48]:
wordCounts = wordsRDD.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print("\nWord count:\n", wordCounts.collect())


Word count:
 [('The', 1035), ('Project', 79), ('of', 8136), ('Ulysses', 6), ('ebook', 2), ('is', 1242), ('use', 42), ('anyone', 17), ('anywhere', 11), ('in', 4612), ('United', 19), ('States', 7), ('other', 271), ('world', 92), ('at', 1215), ('no', 363), ('restrictions', 2), ('whatsoever.', 2), ('may', 71), ('it,', 116), ('give', 121), ('away', 115), ('re-use', 2), ('this', 379), ('online', 4), ('are', 467), ('have', 620), ('check', 5), ('country', 42), ('where', 204), ('before', 182), ('using', 8), ('eBook.', 2), ('Author:', 1), ('date:', 1), ('July', 1), ('1,', 3), ('2003', 1), ('[eBook', 1), ('March', 5), ('Language:', 1), ('English', 19), ('EBook', 1), ('produced', 14), ('***', 4), ('OF', 36), ('GUTENBERG', 3), ('ULYSSES', 2), ('Contents', 1), ('[', 36), ('1', 53), ('II', 2), ('4', 21), ('8', 9), ('9', 14), ('10', 17), ('12', 6), ('14', 9), ('III', 2), ('16', 11), ('17', 5), ('came', 187), ('stairhead,', 1), ('bowl', 11), ('lather', 3), ('mirror', 31), ('razor', 5), ('yellow', 39),

In [ ]:
sc.stop()